In [1]:
# Standard packages
from skimage import morphology

import numpy as np
import pandas as pd

import SimpleITK as sitk
import porespy as ps
from radiomics import featureextractor, shape

from collections import Counter

# Custom packages
import auxiliary.values as v
from auxiliary.data.dataset_ht import HtDataset
from auxiliary.utils.colors import bcolors as c
from auxiliary.data import imaging

In [2]:
ds = HtDataset()
specimens, out_paths = ds.check_features(verbose=1)

Group: Gr1
	Found: 20190521_E2_cell_properties_radiomics.csv
	Found: 20190521_E4_cell_properties_radiomics.csv
	Found: 20190504_E1_cell_properties_radiomics.csv
	Found: 20190806_E5_cell_properties_radiomics.csv
	Found: 20190208_E2_cell_properties_radiomics.csv
Group: Gr2
	Found: 20190209_E1_cell_properties_radiomics.csv
	Found: 20190516_E2_cell_properties_radiomics.csv
	Found: 20190208_E3_cell_properties_radiomics.csv
	Found: 20190520_E2_cell_properties_radiomics.csv
	Found: 20190503_E1_cell_properties_radiomics.csv
Group: Gr3
	Found: 20190308_E3_cell_properties_radiomics.csv
	Found: 20190516_E3_cell_properties_radiomics.csv
	Found: 20190503_E2_cell_properties_radiomics.csv
	Found: 20190209_E2_cell_properties_radiomics.csv
Group: Gr4
	Found: 20190806_E4_cell_properties_radiomics.csv
	Found: 20190523_E1_cell_properties_radiomics.csv
	Found: 20190521_E1_cell_properties_radiomics.csv
	Found: 20190806_E3_cell_properties_radiomics.csv
	Found: 20190806_E6_cell_properties_radiomics.csv
Group:

In [3]:
ds.read_img_paths(type='RawImages', verbose=1)

Raw Images - Nuclei: 49
Raw Images - Membrane: 46


In [4]:
ds.read_img_paths(type='Segmentation', verbose=1)

No directory: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr11/Segmentation/Nuclei
No directory: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr11/Segmentation/Membrane
Segmented - Nuclei: 34
Segmented - Membrane: 31


In [5]:
for s in ['0517_E1']:  # specimens
    print(f'{c.OKBLUE}Specimen:{c.ENDC} {s}')
    
    # path_raw_nuclei, _ = ds.read_specimen(s, level='Nuclei', type='RawImages', verbose=1)
    # raw_nuclei = imaging.read_image(path_raw_nuclei)
    # path_seg_nuclei, _ = ds.read_specimen(s, level='Nuclei', type='Segmentation', verbose=1)
    # seg_nuclei = imaging.read_image(path_seg_nuclei)
    
    path_raw_membrane, _ = ds.read_specimen(s, level='Membrane', type='RawImages', verbose=1)
    raw_membrane = imaging.read_image(path_raw_membrane)

    path_seg_membrane, _ = ds.read_specimen(s, level='Membrane', type='Segmentation', verbose=1)
    seg_membrane = imaging.read_image(path_seg_membrane)
    
    path_line, _ = ds.read_line(s, verbose=1)
    line = imaging.read_image(path_line)

Specimen: 0517_E1
	Found: 20190517_E1_mGFP_decon_0.5.nii.gz
	Found: 20190517_E1_mGFP_XYZ_predictions_GASP.nii.gz
	Found: line_20190517_E1.nii.gz


In [6]:
props_mem = ps.metrics.regionprops_3D(morphology.label(seg_membrane))
centroids_mem = [[round(i) for i in p['centroid']] for p in props_mem]
centroids_labels_mem = [seg_membrane[c[0], c[1], c[2]] for c in centroids_mem]

In [7]:
print(len(centroids_mem), len(centroids_labels_mem))

10834 10834


In [8]:
from multiprocessing import Pool

In [9]:
def compute_most_commons(slice):
    counter = Counter(line[slice].flatten())
    if len(list(counter)) == 1:
        m = list(counter)[0]
    else:
        d = {k: v for k, v in counter.items() if k != 0}
        m = max(d, key=d.get)
    return m

slices_list = [p.slices for p in props_mem]

with Pool() as pool:
    most_commons = pool.map(compute_most_commons, slices_list)